목표

Aruomarker 생성 (기존 코드 활용)

Arucomarker 검출 (기존 코드 활용)

Arucomarker 카메라에서 인식

Arucomarker 상대 위치 인식

Arucomarker 정밀도 수정 연습

In [8]:
import numpy as np
import imutils
import cv2 as cv
import sys

In [9]:
# OpenCV에서 지원하는 ARUCO MARKER의 TYPE에 대한 딕셔너리
ARUCO_DICT = {
	"DICT_4X4_50": cv.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv.aruco.DICT_APRILTAG_36h11
}

In [20]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

capture = cv.VideoCapture(0)
while True:
    ret, frame = capture.read()
    # cv.imshow("VideoFrame", frame)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(frame, (7,6), corners2, ret)
        cv.imshow('VideoFrame', frame)
    else:
        cv.imshow("VideoFrame", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cv.destroyAllWindows()

In [21]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera (objpoints, imgpoints, gray.shape[::-1], None , None )

In [22]:
def pose_esitmation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    '''
    frame - 카메라 또는 비디오에서 읽어온 이미지
    aruco_dict_type - 사용하는 MARKER의 TYPE (예. 앞서 설명한 코드의 ARUCO_DICT[aruco_type])
    matrix_coefficients - 카메라 Calibration 과정 후에 획득한 Intrinsic matrix (내부 행렬)
    distortion_coefficients - 카메라 Calibration 과정 후에 획득한 Distortion coefficients (왜곡 계수)
    return:
    frame - 읽어온 이미지 + MARKER의 축
    '''

    # 카메라 또는 비디오에서 읽어온 이미지를 흑백으로 변환
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # 정의한 ARUCO MARKER의 TYPE을
    # 컴퓨터가 이해할 수 있는 형태의 언어로 변환하는 부분임
    cv.aruco_dict = cv.aruco.Dictionary_get(aruco_dict_type)
    # 검출을 위한 파라미터 설정하는 부분임
    # 일반적으로 default 값을 그대로 사용함
    parameters = cv.aruco.DetectorParameters_create()
    # MARKER 검출
    corners, ids, rejected_img_points = cv.aruco.detectMarkers(gray, cv.aruco_dict,parameters=parameters)
        # cameraMatrix=matrix_coefficients,
        # distCoeff=distortion_coefficients)

    # 하나 이상의 MARKER가 검출됐을 때만 실행되는 부분
    if len(corners) > 0:
        for i in range(0, len(ids)):
            # 각 MARKER의 Pose를 측정 하고 rvec 과 tvec 으로 반환
            rvec, tvec, markerPoints = cv.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                       distortion_coefficients)
            # MARKER 테두리 그려주기 (삭제해도 됨)
            cv.aruco.drawDetectedMarkers(frame, corners) 

            # MARKER 축 그려주기 (삭제해도 됨)
            cv.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)  

    return frame

In [12]:
# 인식할 MARKER의 TYPE을 정의해줌
aruco_type = "DICT_4X4_50"
print("[INFO] detecting '{}' tags...".format(aruco_type))
arucoDict = cv.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

[INFO] detecting 'DICT_4X4_50' tags...


In [23]:
import cv2 as cv

capture = cv.VideoCapture(0)
# capture.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
# capture.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = capture.read()
    # cv.imshow("VideoFrame", frame)
    
    det_frame = pose_esitmation(frame, ARUCO_DICT[aruco_type], mtx, dist)
    cv.imshow("VideoFrame", det_frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cv.destroyAllWindows()
capture.release()